# Imports

In [ ]:
# TODO: check what this is
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

# TODO: check how this affects
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Dataset

In [ ]:


import pathlib
data_dir = pathlib.Path('/content/drive/My Drive/DataSetJeuSet_V1')




image_count = len(list(data_dir.glob('*/*.jpg')))
print(f"le nombre dimage est :{image_count}")
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
print(f'les noms des classes sont : {CLASS_NAMES}')



# Loading the model

In [ ]:
# The 1./255 is to convert from uint8 to float32 in range [0,1].
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                  zoom_range=0.2,
                                                                  validation_split=0.3)


#Loader Parametres
BATCH_SIZE = 32
IMG_SIZE = 160
IMG_HEIGHT = 160
IMG_WIDTH = 160
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES),
                                                     subset='training')

validation_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES),
                                                     subset='validation')

# next(train_data_gen)

Batch inspection

In [ ]:
# INSPECT THE BATCH
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(Taille_BATCH):
      ax = plt.subplot(8,6,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

In [ ]:
image_batch, label_batch = next(train_data_gen)
image_batch.shape
# show_batch(image_batch, label_batch)
# bel_batch.tolist()

## Performance

In [ ]:
import time
default_timeit_steps = 1000

def timeit(ds, steps=default_timeit_steps):
  start = time.time()
  it = iter(ds)
  for i in range(steps):
    batch = next(it)
    if i%10 == 0:
      print('.',end='')
  print()
  end = time.time()

  duration = end-start
  print("{} batches: {} s".format(steps, duration))
  print("{:0.5f} Images/s".format(BATCH_SIZE*steps/duration))

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)
